# Custom Chatbot Project: WSL Surf Results

### Overview
I chose to use data from the World Surf League [rankings](https://www.worldsurfleague.com/athletes/tour/wct?year=2024) and [event](https://www.worldsurfleague.com/events?month=may&year=2024) pages to provide context for surf event result and rankings questions from 2022 to the present using ChatGPT 3.5. Without this context, ChatGPT 3.5 cannot generate a correct response because its training data only goes up until 2021. See [Custom Performance Demonstration](#Custom-Performance-Demonstration) below for examples.

### Process
Experimenting with ChatGPT 4.0, I noticed that even though its last training date was in 2023 it provided accurate answers because it could automatically search the web for up to date information. (I verified this by asking it to explain how it came up with that answer!) Although ChatGPT 4.0 provides a better result than 3.5, there is still a legitimate use case for choosing the older model. Because it is cheaper per token than 4.0, a business case could be made for implementing a custom RAG solution depending on the scale of the application.

## Data Wrangling

Get the 2022 — 2024 WSL rankings and event schedules and results from the WSL Website.

Event request format: `https://www.worldsurfleague.com/events?all=1&year=<year>`

Rankings request format: `https://www.worldsurfleague.com/athletes/tour/<wct|mct>?year=<year>`

In [4]:
from datetime import datetime as dt

import requests
from bs4 import BeautifulSoup
import pandas as pd
from itertools import chain

In [5]:
years = ['2022','2023','2024']

In [6]:
def get_rankings_by_year(year):
    genders = ['mct','wct']
    data = []
    for gender in genders:
        res = requests.get(f"https://www.worldsurfleague.com/athletes/tour/{gender}?year={year}")
        soup = BeautifulSoup(res.text)
        rankings = soup.find_all('a', class_='athlete-name')
        gender_map = {"mct": "male", "wct": "female"}
        data.append([{"year": year, "gender": gender_map.get(gender), "rank": rank + 1, "name": el.text} for rank, el in enumerate(rankings)])
    return data

In [7]:
results = []
for year in years:
    data = get_rankings_by_year(year)
    results.append(data)

In [8]:
df = pd.DataFrame(chain(*chain(*results)))

In [9]:
df[["first_name", "last_name"]] = df['name'].str.split(' ', n=1, expand=True)

In [10]:
df = df.replace("Joao","João").replace("Joao Chianca", "João Chianca")

In [11]:
df["text"] = df.apply(lambda x: f"{x['name']} {'is' if int(x.year) == dt.now().year else 'was' } the number {x['rank']} ranked {x.gender} surfer in {x.year}".lower(), axis=1)

In [12]:
df_athletes = df[["name","gender"]].drop_duplicates().reset_index(drop=True)

In [13]:
import re
from daterangeparser import parse
def get_events_by_year(year):
    data = []
    res = requests.get(f"https://www.worldsurfleague.com/events?all=1&year={year}")
    soup = BeautifulSoup(res.text)
    event_pattern = re.compile(r"^event-*")
    event_table_rows = soup.find("div", class_="events-schedule-table").find_all("tr", class_=event_pattern)
    return event_table_rows, year

def parse_event_cols(rows, year):
    result = []
    for row in rows:
        date_range, location, tour, status = row.find_all("td")
        event_details = location.find("a")
        event_link = event_details["href"] if event_details else ""
        event_data = eval(event_details["data-gtm-event"]) if event_details else ""
        tour_code = event_data["tour_code"] if event_details else ""
        event_name = event_data["event_name"] if event_details else ""
        start, end = parse(f'{date_range.text} {year}')
        if tour_code and tour_code.lower() == "mct":
            result.append({
                "start_date": start,
                "end_date": end,
                "year": year,
                "event_name": event_name,
                "tour_code": tour_code,
                "event_link": event_link.strip(),
                "status": status.text.lower().strip()
            })
    return result

In [14]:
results = []

for year in years:
    row_data, year = get_events_by_year(year)
    result = parse_event_cols(row_data, year)
    results.append(result)
df_events = pd.DataFrame(chain(*results))

### Extract Links from Contest Results

In [15]:
df_events["id"] = pd.util.hash_pandas_object(df_events, index=False)

In [16]:
def get_event_by_url(url) -> list:
    result = []
    url = url.replace("/main","/results")
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    champs = soup.find("span", class_="status-module__status-message")
    return champs

def parse_event_results(data, comp_id: str, athletes: pd.DataFrame):
    results = []
    for _, row in athletes.iterrows():
        if row["name"] in data.text:
            results.append({"comp_id": comp_id, "athlete_name": row["name"], "athlete_gender": row["gender"]})
    return results
        

In [17]:
comp_results = []

for idx, e in df_events[df_events["end_date"] < dt.now()].iterrows():
    data = get_event_by_url(e.event_link)
    results = parse_event_results(data, e.id, df_athletes)
    comp_results.append(results)

In [18]:
df_athletes[df_athletes["name"].isin(["João Chianca"])]

,name,gender
33,João Chianca,male


In [19]:
comp_results_df = pd.DataFrame(list(chain(*comp_results)))

In [20]:
comp_results_df[comp_results_df["athlete_gender"] == "male"].sort_values("comp_id").shape

(26, 3)

In [21]:
comp_results_df = comp_results_df.merge(df_events, left_on="comp_id", right_on="id").drop(["event_link", "tour_code", "id", "comp_id"], axis=1)

In [22]:
comp_results_df["text"] = comp_results_df.apply(lambda x: f"{x.athlete_name} won the {'mens' if x.athlete_gender == 'male' else 'womens'} {x.year} {x.event_name}".lower(), axis=1)

In [23]:
all_df = pd.DataFrame({"text": pd.concat([comp_results_df["text"], df["text"]])})

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [26]:
import os
import openai as client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [27]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 10
embeddings = []
for i in range(0, len(all_df), batch_size):
    response = client.embeddings.create(
        input=all_df.iloc[i:i+batch_size]["text"].tolist(),
        model=EMBEDDING_MODEL_NAME
    )
    embeddings.extend([data.embedding for data in response.data])
all_df["embeddings"] = embeddings

In [28]:
from scipy.spatial.distance import cosine as cosine_distance

def get_rows_sorted_by_relevance(question, df):
    question_embeddings = client.embeddings.create(
        input=question,
        model=EMBEDDING_MODEL_NAME
    )
    df_copy = df.copy()
    df_copy["distances"] = df_copy["embeddings"].apply(lambda row: cosine_distance(row, question_embeddings.data[0].embedding))
    df_copy = df_copy.sort_values("distances", ascending=True).reset_index(drop=True)
    return df_copy

In [29]:
USER_QUESTION = """Who won the womens 2023 Pipe Pro?"""

In [87]:
import tiktoken

def create_prompt(question, df, max_token_count):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [88]:
question_01 = "Who was the number 1 ranked female surfer in 2024 in the WSL?"

In [89]:
def generic_prompt(question):
    return f"""
Question: {question_01}
Answer:
"""

In [90]:
surf_prompt = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=generic_prompt(question_01),
    stream=False,
    max_tokens=150
)
print(surf_prompt.choices[0].text)


It is not possible to determine who the number 1 ranked female surfer will be in 2024 as it is a future event and can be influenced by many factors.


In [91]:
prompt = create_prompt(question_01, all_df, 500)
initial_surf_answer = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=500,
    temperature=0
)
print(initial_surf_answer.choices[0].text.strip())

Caitlin Simmers


**Caitlin Simmers** is the correct response 👏🏼

### Question 2

In [92]:
question_02 = "Who won the Pipe Pro surfing event in 2023?"

In [93]:
surf_prompt = f"""
Question: {question_02}
Answer:
"""
surf_prompt = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=surf_prompt,
    stream=False,
    max_tokens=150
)
print(surf_prompt.choices[0].text.strip())

I'm sorry, I cannot accurately predict future events.


In [94]:
prompt = create_prompt(question_02, all_df, 250)
initial_surf_answer = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=250,
    temperature=0
)
print(initial_surf_answer.choices[0].text.strip())

Jack Robinson won the mens 2023 billabong pro pipeline.


**Jack Robinson** is correct, but the result should also include Carissa Moore. While the initial result included both surfers, retrying the code only yielded the male surfer. Unfortunately, this looks like gender bias in the model.

In [95]:
question_03 = "Who won the womens' Pipe Pro surfing event in 2023?"

In [96]:
prompt = create_prompt(question_03, all_df, 250)
initial_surf_answer = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=250,
    temperature=0
)
print(initial_surf_answer.choices[0].text.strip())

Carissa Moore


**Carissa More** is correct here. 